# Datengrundlage zur Sentiment-Analyse

In [16]:
from germansentiment import SentimentModel
import pandas as pd

Für die Sentiment-Analyse wird das Python-Package `germansentiment` verwendet. Die Textanalyse für englisch-sprachige Texte ist deutlich ausgereifter als die für deutsche Texte. Das führt insbesondere bei der Sentiment-Analyse zu problemen, da hier die Bedeutung der Worte zur Verfügung stehen muss. Das zuvor genannte Package macht es jedoch auch möglich deutsche Texte zu analysieren.

Wir lesen zunächst die bereits bestehenden Daten mit den Artikeln ein.

In [18]:
articles_data = pd.read_csv('2-3-data-extended-keywords.csv')
articles = pd.DataFrame(articles_data)
articles.head()

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,"Politik, Gesellschaft, Extremismus, Rechtsextr..."
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,"China, USA, Paracel-Inseln, Südchinesisches Me..."
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,"Politik, Gewalt gegen Politiker, SPD, Wahlkamp..."
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,"Bürgergeld, Landkreistag, Vermögensregelungen,..."
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,"Regierungsflugzeug, Technikproblem, Verzögerun..."


## Sentiment-Bestimmung

Das Dataframe wird nun um die Ergebnisse der Sentiment-Analyse erweitert. Dazu werden jeweils die Spalten `headline` und `text` des Dataframes extrahiert und anschließend dem `SentimentModel` übergeben. Zurückgegeben wird sowohl die entsprechende Bewertung als auch die einzelnen Wahrscheinlichkeiten für die jeweilige Klassifizierung. Dadurch können die Ergebnisse sowohl allein für die Überschrift als auch für den Text bewertet und verglichen werden.

In [ ]:
sentiment_model = SentimentModel()

### Sentiment der Überschriften

In [6]:
headlines = articles['headline']
headlines_classes, headlines_probabilities = sentiment_model.predict_sentiment(headlines, output_probabilities = True)

In [7]:
articles = articles.assign(sentiment_headline=headlines_classes)
articles = articles.assign(sentiment_prob_headline=headlines_probabilities)
articles.head()

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords,sentiment_headline,sentiment_prob_headline
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,"Politik, Gesellschaft, Extremismus, Rechtsextr...",neutral,"[[positive, 0.00022852589609101415], [negative..."
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,"China, USA, Paracel-Inseln, Südchinesisches Me...",negative,"[[positive, 0.02673402987420559], [negative, 0..."
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,"Politik, Gewalt gegen Politiker, SPD, Wahlkamp...",negative,"[[positive, 0.005901369731873274], [negative, ..."
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,"Bürgergeld, Landkreistag, Vermögensregelungen,...",neutral,"[[positive, 9.093517292058095e-06], [negative,..."
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,"Regierungsflugzeug, Technikproblem, Verzögerun...",neutral,"[[positive, 8.408874418819323e-05], [negative,..."


### Sentiment der Artikel

In [8]:
def sentiment_on_texts(articles_df):
    length = len(articles_df)
    sentiments = []
    for index, row in articles_df.iterrows():
        print(f'\rRunning sentiment analysis on article {index + 1} of {length}', end='')
        text_class, text_probabilities = sentiment_model.predict_sentiment([row["text"]], output_probabilities = True)
        sentiments.append((text_class, text_probabilities))
    return sentiments

In [9]:
text_sentiments = sentiment_on_texts(articles)

Running sentiment analysis on article 10678 of 10678

In [10]:
texts_classes = [x[0][0] for x in text_sentiments]
texts_probabilities = [x[1][0] for x in text_sentiments]
articles["sentiment_text"] = texts_classes
articles["sentiment_prob_text"] = texts_probabilities
articles

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords,sentiment_headline,sentiment_prob_headline,sentiment_text,sentiment_prob_text
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,"Politik, Gesellschaft, Extremismus, Rechtsextr...",neutral,"[[positive, 0.00022852589609101415], [negative...",neutral,"[[positive, 0.0019495805026963353], [negative,..."
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,"China, USA, Paracel-Inseln, Südchinesisches Me...",negative,"[[positive, 0.02673402987420559], [negative, 0...",neutral,"[[positive, 0.0028206082060933113], [negative,..."
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,"Politik, Gewalt gegen Politiker, SPD, Wahlkamp...",negative,"[[positive, 0.005901369731873274], [negative, ...",negative,"[[positive, 0.019161105155944824], [negative, ..."
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,"Bürgergeld, Landkreistag, Vermögensregelungen,...",neutral,"[[positive, 9.093517292058095e-06], [negative,...",neutral,"[[positive, 0.0020323987118899822], [negative,..."
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,"Regierungsflugzeug, Technikproblem, Verzögerun...",neutral,"[[positive, 8.408874418819323e-05], [negative,...",negative,"[[positive, 0.01217340026050806], [negative, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10673,2024-06-11,17:09:00,wirtschaft,Rheinmetall eroeffnet Reparaturbetrieb in Ukraine,Rheinmetall ist in der Ukraine schon länger ak...,2024-06-11 17:09:00,17,Tuesday,2724,"Rheinmetall, Waffenproduktion, Ukraine, Panzer...",neutral,"[[positive, 0.0008944926667027175], [negative,...",neutral,"[[positive, 0.0001203527717734687], [negative,..."
10674,2024-06-11,11:00:00,wirtschaft,Vermoegen der RAG Stiftung steigt auf 18 Milli...,Um die Kosten für das Ende des Steinkohlebergb...,2024-06-11 11:00:00,11,Tuesday,2943,"RAG-Stiftung, Steinkohle, Ewigkeitskosten, Ene...",neutral,"[[positive, 1.1808124327217229e-05], [negative...",neutral,"[[positive, 7.319341875700047e-06], [negative,..."
10675,2024-06-11,09:58:00,wirtschaft,Fuest AfD Erfolg ist Gefahr fuer Wirtschaft im...,In den Wahlerfolgen von AfD und BSW sieht der ...,2024-06-11 09:58:00,9,Tuesday,2785,"Politik, Wirtschaft, AfD, Wahl, Ostdeutschland...",neutral,"[[positive, 0.033423688262701035], [negative, ...",negative,"[[positive, 0.034321654587984085], [negative, ..."
10676,2024-06-11,22:11:00,wirtschaft,KI Firma Mistral sichert sich naechste Finanzs...,Mistral ist ein vielversprechender europäische...,2024-06-11 22:11:00,22,Tuesday,2400,"Mistral, Generative KI, Investoren, Finanzieru...",neutral,"[[positive, 0.00011987510515609756], [negative...",neutral,"[[positive, 6.636761099798605e-05], [negative,..."


Die einzelnen Wahrscheinlichkeiten werden in einzelne Spalten aufgeteilt, damit sie anschließend in einer `csv`-Datei gespeichert werden können.

In [11]:
articles["sentiment_prob_headline_positive"] = articles.apply(lambda row: row["sentiment_prob_headline"][0][1], axis=1)
articles["sentiment_prob_headline_negative"] = articles.apply(lambda row: row["sentiment_prob_headline"][1][1], axis=1)
articles["sentiment_prob_headline_neutral"]  = articles.apply(lambda row: row["sentiment_prob_headline"][2][1], axis=1)

In [12]:
articles["sentiment_prob_text_positive"] = articles.apply(lambda row: row["sentiment_prob_text"][0][1], axis=1)
articles["sentiment_prob_text_negative"] = articles.apply(lambda row: row["sentiment_prob_text"][1][1], axis=1)
articles["sentiment_prob_text_neutral"]  = articles.apply(lambda row: row["sentiment_prob_text"][2][1], axis=1)

Die vorherigen Spalten können daher gelöscht werden.

In [13]:
articles = articles.drop('sentiment_prob_headline', axis=1)
articles = articles.drop('sentiment_prob_text', axis=1)

In [14]:
articles.head()

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords,sentiment_headline,sentiment_text,sentiment_prob_headline_positive,sentiment_prob_headline_negative,sentiment_prob_headline_neutral,sentiment_prob_text_positive,sentiment_prob_text_negative,sentiment_prob_text_neutral
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,"Politik, Gesellschaft, Extremismus, Rechtsextr...",neutral,neutral,0.000229,0.000604,0.999167,0.001950,0.010997,0.987053
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,"China, USA, Paracel-Inseln, Südchinesisches Me...",negative,neutral,0.026734,0.750409,0.222857,0.002821,0.010704,0.986475
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,"Politik, Gewalt gegen Politiker, SPD, Wahlkamp...",negative,negative,0.005901,0.979033,0.015065,0.019161,0.757548,0.223291
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,"Bürgergeld, Landkreistag, Vermögensregelungen,...",neutral,neutral,0.000009,0.000026,0.999965,0.002032,0.010209,0.987759
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,"Regierungsflugzeug, Technikproblem, Verzögerun...",neutral,negative,0.000084,0.000253,0.999663,0.012173,0.830547,0.157280


In [15]:
articles.to_csv('3-4-data-extended-keywords-sentiment.csv', sep=',', encoding='utf-8', index=False)

# Topic Modeling

Für die Modellierung der Themen wird `BERTopic` verwendet. Dafür muss gegebenfalls noch nachfolgender Befehl ausgeführt werden. Anschließend kann die Bibliothek import werden.
- `pip install bertopic`

## Einlesen der Daten

Wie zuvor werden zunächst die Daten des vorherigen Schrittes eingelesen. Das ermöglicht das Überspringen des vorherigen Codes.

In [ ]:
import pandas as pd

In [ ]:
articles_data = pd.read_csv('2-2-sentiment.csv')
articles = pd.DataFrame(articles_data)
articles

## Ausführen des Topic Modeling

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

Erstellt wird das `topic_model`, das eine Reihe an Sprachen verarbeiten kann und übergeben diesem die Überschriften der Artikel. Wir beschränken die Anzahl der Topics, die gefunden werden sollen auf `60`. Wird keine Begrenzung gesetzt, also das Argument `nr_topics=<int>` entfernt, so kann in der Regel der erste Eintrag ignoriert werden, da dieser ausschließlich Füllwörter enthält.

In [ ]:
representation_model = KeyBERTInspired()
topic_model = BERTopic(language="multilingual", representation_model=representation_model, nr_topics=60)
topic_model.fit_transform(articles["headline"])
topic_result = topic_model.get_topic_info()

In [ ]:
topic_result

## Speicherung

Auch diese Ergebnisse werden in einer `csv`-Datei gespeichert. Zu beachten ist hier, dass sich die Ergebnisse bei jeder Ausführung unterscheiden und daher von den derzeit gespeicherten Ergebnissen abweichen können.

In [ ]:
topic_result.to_csv('2-4-data-extended-keyword-sentiment.csv', sep=',', encoding='utf-8', index=False)

# Analyse

Als nächstes sollen die Ergebnisse untersucht werden. Dazu wird das Package `matplotlib` verwendet, das mit folgendem Befehl installiert werden kann:
- `pip install matplotlib`

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

Zunächst werden die Ergebnisse geladen.

In [4]:
articles_data = pd.read_csv('2-2-sentiment.csv')
articles = pd.DataFrame(articles_data)
topics_data = pd.read_csv('2-3-topics.csv')
topics = pd.DataFrame(topics_data)

Anschließend wird eine Funktion definiert, die die Ergebnisse der Sentiment-Analyse visualisiert.

In [5]:
def create_sentiment_result_plot(df, column):
    sentiment_count = []
    for sentiment_class, count in df[column].value_counts().items():
        sentiment_count.append( (sentiment_class, count) )
    labels, values = zip(*sentiment_count)
    plt.xlabel('Sentiment class')
    plt.ylabel('Count')
    plt.title('Sentiment analyse of headlines')
    plt.bar(labels, values)
    for i, value in enumerate(values):
        print(i, value)
        plt.text(i, value + 0.1, str(value), ha='center', va='bottom')
    plt.show()

Zu sehen ist, dass ein Großteil der Nachrichten als `neutral` klassifiziert wurden, was für einen neutralen Nachrichtensender entsprechend wünschenswert ist. Dennoch wurde auch ein nicht zu vernachlässigenderer Teil der Überschriften als `negativ` klassifiziert. Positive Überschriften gibt es hingegend nur sehr wenige. Betrachtet man nicht die Überschriften, sondern die Texte, ist ein noch größerer Teil der Überschriften `neutral` und so gut wie keine positiven Artikel.